In [29]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from mlxtend.classifier import StackingClassifier
import numpy as np
import pandas as pd
import pickle

In [33]:
#clf1 = LinearSVC(dual=False)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=20, max_features=10000, n_jobs=-1)
clf3 = MultinomialNB()#GaussianNB()
lr = LogisticRegression(solver='sag')
sclf = StackingClassifier(classifiers=[clf2, clf3], 
                          meta_classifier=lr)

In [20]:
train = pickle.load(open('train.p','rb'))
test = pickle.load(open('test.p','rb'))
y_train=train.iloc[:, 1:]

In [23]:
train = pickle.load(open('train.p','rb'))
test = pickle.load(open('test.p','rb'))

train_text = train['comment_text']
test_text = test
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=40000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
print('word TFIDF')

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=40000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)
print('char TFIDF')



word TFIDF
char TFIDF


NameError: name 'hstack' is not defined

In [24]:
from scipy.sparse import hstack
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [34]:
submission = pd.DataFrame.from_dict({'id': test.index.tolist()})
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for class_name in class_names:
    train_target = train[class_name]
    print("training on ",class_name)
    sclf.fit(train_features, train_target)
    print("classification on ",class_name)
    submission[class_name] = sclf.predict_proba(test_features)[:, 1]
print('##################################')
submission.to_csv('Stacking_lr_rndmfrst_char_word.csv', index=False)

training on  toxic
classification on  toxic
training on  severe_toxic
classification on  severe_toxic
training on  obscene
classification on  obscene
training on  threat
classification on  threat
training on  insult
classification on  insult
training on  identity_hate
classification on  identity_hate
##################################
